In [2]:
import sys
from datasets import load_dataset
import torch
from trl.trainer.dpo_trainer import DPOTrainer
import gc

sys.path.append('..')
from model.causal_lm import AutoLMFineTuner
from model.utils import LMHyperParams, SmModel, ModelChoice
from train import start_training
from dataset.squad import UltraFeedbackDataModule
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
params = LMHyperParams(
    base_model_checkpoint="Qwen/Qwen2.5-0.5B",
    warmup_steps_count=10,
    tuning_type="dpo",
    train_batch_size=1,
    gradient_accumulation_steps=1,
    optimizer="AdamW",
)

# https://github.com/philschmid/deep-learning-pytorch-huggingface/blob/main/training/dpo-align-llms-in-2024-with-trl.ipynb

In [4]:
gc.collect()
torch.cuda.empty_cache()

In [5]:
model = AutoLMFineTuner(params)
model.to('cuda')

2024-11-18 02:55:23.760 | INFO     | model.causal_lm:__init__:69 - Using DPO with LoraConfig: LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='CAUSAL_LM', inference_mode=False, r=256, target_modules='all-linear', lora_alpha=128, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))


AutoLMFineTuner(
  (model): PeftModelForCausalLM(
    (base_model): LoraModel(
      (model): Qwen2ForCausalLM(
        (model): Qwen2Model(
          (embed_tokens): Embedding(151936, 896)
          (layers): ModuleList(
            (0-23): 24 x Qwen2DecoderLayer(
              (self_attn): Qwen2SdpaAttention(
                (q_proj): lora.Linear(
                  (base_layer): Linear(in_features=896, out_features=896, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=896, out_features=256, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=256, out_features=896, bias=False)
                  )
                  (lora_embedding_A): ParameterDict()
                  (lora_embedding_B): ParameterDict()
                  (lora_magnitude_vec

In [22]:
data_module = UltraFeedbackDataModule(2, model.tokenizer, 1024, 1000)
data_module.cpu_count = 12
data_module.setup("fit")

2024-11-18 03:18:06.733 | INFO     | dataset.squad:setup:219 - Loading dataset for stage fit


2024-11-18 03:18:07.820 | INFO     | dataset.squad:setup:230 - Processing dataset for stage fit, workers: 12, cache dir dataset_caches/ultrafeedback
Map (num_proc=12): 100%|██████████| 100/100 [00:01<00:00, 68.37 examples/s]


In [23]:
data_module.precompute_reference_logprobs(model)

Precomputing reference logprobs:   0%|          | 0/450 [00:00<?, ?it/s]

Precomputing reference logprobs: 100%|██████████| 450/450 [01:22<00:00,  5.47it/s]


ArrowInvalid: Could not convert tensor([[ 8.9819,  8.6127, 13.7972,  ..., -2.2760, -2.2768, -2.2758],
        [ 9.2672,  9.5326, 13.2523,  ..., -2.0715, -2.0724, -2.0714],
        [ 9.7109,  9.5706, 13.3365,  ..., -2.1283, -2.1291, -2.1282],
        ...,
        [ 4.7126, -1.4167,  6.5034,  ..., -3.4425, -3.4431, -3.4427],
        [ 9.8718,  9.5079, 10.7250,  ..., -2.9143, -2.9143, -2.9144],
        [-5.2703, -5.2136, -3.4976,  ...,  7.1035,  7.1041,  7.1034]]) with type Tensor: did not recognize Python value type when inferring an Arrow data type

In [ ]:
for batch in data_module.train_dataloader():
    for k, v in batch.items():
        batch[k] = v.to('cuda')
    loss = model._step(batch)
    print(loss)
    break

In [ ]:
start_training(
    "16-mixed", "", model, model.tokenizer, data_module, params, False, ""
)
